This is the code for the Supply chain Project. This project was created for the class "Συνδυαστική και γραμμική βελτιστοπποίηση".

At first we import the needed libraries.

In [97]:
import numpy as np 
import pymprog as py
import pymprog
import pulp
import traceback


Then we add the variables as numpy arrays needed to solve the problem
This section will represent the imput values

In [48]:
T = 10 #planning horizon [-τi, T]
n = 11
pi = np.array([2,4,1,1,2,3,2,2,1,1,3])  #unit cost in terms of capacity 
Hij = np.zeros((n,n))
Hij[2,0] = 2
Hij[3,0] = 3
Hij[4,1] = 3
Hij[5,1] = 2
Hij[6,2] = 1
Hij[7,2] = 2
Hij[8,3] = 4
Hij[9,5] = 1
Hij[10,4] = 2

Ti = np.array([2,2,2,2,2,2,2,2,2,2,2]) #the planned lead time of item i, due to the high complexity, we chose to use only a single lead time for all the items

ait = np.array([[1,1,2,1,1,2,1,2,1,2,2],
                [2,2,1,1,5,3,4,3,2,1,2],
                [2,4,3,2,1,3,1,1,2,4,1],
                [3,4,1,2,2,2,1,2,1,1,2],
                [2,1,2,2,1,1,1,1,2,4,3],
                [4,3,2,1,1,2,2,2,1,2,1],
                [4,2,2,1,2,3,1,2,2,2,1],
                [3,2,1,2,5,1,1,1,1,1,1],
                [2,3,1,4,3,2,2,3,1,1,1],
                [2,1,1,2,2,1,3,4,1,2,1]])   #the inventory costs

bit = np.array([[1,1,0,1,1,2,1,2,1,2,2],
                [2,0,1,1,5,3,0,0,2,0,2],
                [2,0,3,2,0,0,1,0,2,0,1],
                [3,0,0,2,2,2,1,0,1,0,2],
                [2,1,0,2,0,1,0,1,2,4,3],
                [4,0,0,1,1,0,2,2,1,0,1],
                [0,2,2,0,0,3,1,0,0,2,1],
                [3,2,1,2,5,1,1,0,0,1,1],
                [2,3,0,0,3,0,2,3,1,0,1],
                [2,1,1,0,2,1,3,0,1,2,1]])  # the costs due to backordering

Dit = np.array([[1,1,2,1,1,2,1,2,1,2,2],
                [2,2,0,0,0,0,0,3,2,1,2],
                [2,0,3,2,1,3,1,0,0,0,1],
                [3,4,0,0,0,0,0,0,1,1,2],
                [2,1,2,2,1,1,1,1,2,4,3],
                [4,0,0,1,1,2,2,2,0,2,1],
                [4,0,2,1,2,0,1,2,0,2,1],
                [3,0,1,0,5,1,1,1,0,1,1],
                [2,3,1,0,3,0,0,3,0,1,1],
                [2,1,1,0,2,1,3,4,1,2,1]])   #the exogeneously determined demand

ki = np.array([1,2,3]) #the number of resources

k = 3

cut = np.array([[3,1,3],
               [2,2,1],
               [1,2,1],
               [1,2,1],
               [2,1,3],
               [3,2,2],
               [1,2,1],
               [2,3,1],
               [2,3,1],
               [1,2,2]]) #the max production time available on resource u in time slot t

Ri = np.array([1,2,1,2,3,3,3,1,2,1,2]) #  the set of resources that are used by i 

Iit = np.zeros((n, T - 1), dtype=object)

#Iit[0] = np.array([[0,0,0,1,1,0,0,1,0,1,1]]) #the inventory of item i at time -1

Bit = np.zeros(( n,T - 1), dtype=object)
#Bit[0] = np.array([[0,0,0,1,0,0,0,1,1,0,0]]) #the backorders of item i at time -1

Rit = np.zeros((n, (int(2) + T - 1 ) ), dtype=object)# the planned work order release of item i already determined in the past. We use this for time [-Ti,0], since we will use the same array in the furture
print(Rit)

Viut = np.zeros((n, k, int(2) - 2 + T ), dtype=object) #the capacity alreasy allocated to an item at resource u in time slot t.We use this for time [-Ti+2,0], since we will use the same array in the furture

Zits = np.zeros((n, int(2) - 1 + T - 1 , T ), dtype=object) #the part of planned work order release of item i requested on at time t and already executed in time slot s

Git = np.zeros((n, T - 1), dtype=object) #the gross requirements of item i at time t

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]


Now that we have initialized our input values we can proceed to the next step that is to add the decision variables needed for our problem. We already have added them, Iit, Bit, Rit, Viut, Zits during the previous step because they needed initialization, at time -1. 

In [143]:
import numpy as np 
import pymprog as py
import pymprog
import pulp



model = pulp.LpProblem('SCOP', pulp.LpMinimize)   # begin modeling

# Variables
x = np.zeros((n, T+2), dtype=object)  # Create a matrix of variables
for i in range(n):
    for j in range(T+2):
        x[i, j] = pulp.LpVariable(f'x_{i}_{j}', lowBound=0, cat='Continuous')   # Create variables

# Additional variables
I = np.zeros((T, n), dtype=object)  # Create a matrix of auxiliary variables
for t in range(T-1):
    for i in range(n):
        I[t, i] = pulp.LpVariable(f'I_{t}_{i}', lowBound=0, cat='Continuous')   # Create auxiliary variables

G = np.empty((T - 1, n), dtype=object) 
for t in range(T-1):
    for i in range(n):
        G[t, i] = pulp.LpVariable(f'G_{t}_{i}', lowBound=0, cat='Continuous')   # Create auxiliary variables

B = np.empty((T - 1, n), dtype=object) 
for t in range(T-1):
    for i in range(n):
        B[t, i] = pulp.LpVariable(f'B_{t}_{i}', lowBound=0, cat='Continuous')   # Create auxiliary variables

V = np.empty((T - 2 + 2, n, max(Ri) + 1), dtype=object) 
for t in range(T - 2 + 2):
    for i in range(n):
        for u in range(max(Ri) + 1):
            V[t, i, u] = pulp.LpVariable(f'V_{t}_{i}_{u}', lowBound=0, cat='Continuous')   # Create auxiliary variables

Z = np.empty((- 2 + 1 + T, n, T), dtype=object) 
for t in range(T - 1 ):
    for i in range(n):
        for s in range(T):
            Z[t, i, s] = pulp.LpVariable(f'Z_{t}_{i}_{s}', lowBound=0, cat='Continuous')   # Create auxiliary variables

# Objective
    model += pulp.lpSum([pulp.lpSum([ait[t,i]*I[t-1, i] for i in range(n)]) for t in range(T)]) + pulp.lpSum([pulp.lpSum([bit[t,i]*B[t-1, i] for i in range(n)]) for t in range(T)]) , 'obj'

# Constraints
for t in range(T-1):
    for i in range(n):
        if t > 0:
            constraint_expr2 = I[t, i] == I[t - 1, i] + x[i, t - 2] - Dit[t,i] - G[t,i] + B[t, i] - B[t-1, i] 
        else:
            constraint_expr2 = I[t, i] == x[i, t - 2] - Dit[t,i] - G[t,i] + B[t, i]
        model.addConstraint(constraint_expr2)

for t in range(T - 1):
    for i in range(n):
        constraint_expr3 = G[t, i] == pulp.lpSum(Hij[i, j] * x[j, t] for j in range(n))
        model.addConstraint(constraint_expr3)

for i in range(n):
    for t in range(0, T - 1):
        constraint_expr4 = pulp.lpSum(pi[i] * x[i, s] for s in range(-2, t)) >= pulp.lpSum(
            pulp.lpSum(V[s, i, u] for s in range(-1, t + 1)) for u in Ri)
        model.addConstraint(constraint_expr4)

for i in range(n):
    for t in range(0, T - 2):
        constraint_expr5 = pulp.lpSum(pi[i] * x[i, s] for s in range(-2, t)) <= pulp.lpSum(
            pulp.lpSum(V[s, i, u] for s in range(-1, t + 2)) for u in Ri)
        model.addConstraint(constraint_expr5)

for t in range(T):
    for u in range(k):
        constraint_expr6 = pulp.lpSum(V[t, i, u] for i in range(n)) <= cut[t][u]
        model.addConstraint(constraint_expr6)

for t in range(T - 1):
    for i in range(n):
        constraint_expr7 = B[t, i] - B[t-1, i] <= Dit[t,i]
        model.addConstraint(constraint_expr7)


# Solve the model
model.solve()

# Access variable values
x_values = np.zeros((n, T))
for i in range(n):
    for j in range(T):
        x_values[i, j] = x[i, j].value()

print("Problem Status:", pulp.LpStatus[model.status])

print(x_values)

Problem Status: Optimal
[[ 2.  3.  2.  4.  4.  3.  2.  0.  0. nan]
 [ 0.  4.  1.  0.  0.  0.  3.  0.  0. nan]
 [ 7.  8. 10.  6.  6.  1.  1.  0.  0. nan]
 [ 8. 12. 14. 10.  7.  0.  0.  0.  0. nan]
 [ 4.  0.  1.  1. 11.  5.  3.  0.  0. nan]
 [ 5.  0.  1.  2.  6.  1.  0.  0.  0. nan]
 [11.  6.  7.  3.  2.  1.  0.  0. nan nan]
 [20. 12. 13.  4.  4.  1.  3.  2. nan nan]
 [56. 41. 30.  0.  0.  0.  0. 12. nan nan]
 [ 1.  3. 10.  3.  2.  1.  1.  0. nan nan]
 [ 3.  4. 25. 11.  7.  1.  1.  0. nan nan]]
